Ref: http://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html#sphx-glr-auto-examples-applications-plot-topics-extraction-with-nmf-lda-py

In [1]:
from __future__ import print_function
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [2]:
import sys, os
os.chdir('../data/otu_tables_wTax')

import biom
from biom import parse_table, load_table
fname = 'ITS_otu_table_wTax.biom'
with open(fname) as f: 
    table = parse_table(f)
    

# randomly subsample without replacement
num_sample = 100
samp_table = table.subsample(num_sample,axis='sample',by_id=True)
df = samp_table.to_dataframe()
df.head()

,574.O,1114.O,985.O,1112.O,700.O,264.O,410.O,29.O,22.O,798.O,...,1617.O,1489.O,807.O,1264.O,1180.O,1494.O,1480.O,1599.O,1604.O,1329.O
OTU_360,0.0,0.0,0.0,0.0,0.0,57.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
OTU_5,679.0,2245.0,420.0,92555.0,19.0,16.0,16.0,4053.0,950.0,19.0,...,10.0,5.0,11.0,1.0,102.0,1.0,66.0,54.0,2330.0,39.0
OTU_47557,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OTU_45150,0.0,0.0,28.0,261.0,0.0,1.0,0.0,0.0,432.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OTU_3,20.0,9901.0,38.0,3685.0,25.0,14.0,6.0,10.0,33480.0,2800.0,...,1083.0,0.0,578.0,1.0,25.0,1.0,45.0,378.0,1842.0,137.0


In [3]:
otus = [otu for otu in samp_table.ids(axis='observation')] # 'words'
samples = [samp for samp in samp_table.ids()] # 'documents'

In [4]:
otu_ids = list(xrange(len(otus)))  # indices for 'words' in the dictionary
samp_ids = list(xrange(len(samples))) # indices for 'documents' in the corpus

In [5]:
from numpy import array
otu_ids = array(otu_ids)
samp_ids = array(samp_ids)

In [6]:
print(type(otu_ids))

<type 'numpy.ndarray'>


In [7]:
df.values

array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  6.79000000e+02,   2.24500000e+03,   4.20000000e+02, ...,
          5.40000000e+01,   2.33000000e+03,   3.90000000e+01],
       [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       ..., 
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00]])

In [8]:
print(type(df.values))

<type 'numpy.ndarray'>


In [9]:
df.values.size

3004000

In [10]:
m=len(otus)  # number of 'features' or 'words'
m

30040

In [11]:
n=len(samples)  # number of 'samples' or 'articles'
n

100

In [12]:
n*m

3004000

In [13]:
temp = df.values.reshape((n,m))
print(temp.size)

3004000


In [14]:
import numpy as np
from scipy.sparse import csr_matrix
# construct at compressed sparse row format matrix, to represent the 'tf' matrix
tf = csr_matrix(df.values.reshape((n,m))) #.toarray()
#csr_matrix((df.values.reshape((n,m)), (samp_ids,otu_ids))) #.toarray()

In [15]:
tf

<100x30040 sparse matrix of type '<type 'numpy.float64'>'
	with 176369 stored elements in Compressed Sparse Row format>

In [16]:
print(tf)

  (0, 5)	57.0
  (0, 19)	90.0
  (0, 20)	7.0
  (0, 25)	67.0
  (0, 29)	86.0
  (0, 36)	1.0
  (0, 38)	1.0
  (0, 44)	4.0
  (0, 51)	1.0
  (0, 54)	1.0
  (0, 59)	15.0
  (0, 66)	144.0
  (0, 95)	1.0
  (0, 100)	679.0
  (0, 101)	2245.0
  (0, 102)	420.0
  (0, 103)	92555.0
  (0, 104)	19.0
  (0, 105)	16.0
  (0, 106)	16.0
  (0, 107)	4053.0
  (0, 108)	950.0
  (0, 109)	19.0
  (0, 110)	2523.0
  (0, 111)	13.0
  :	:
  (99, 27734)	5.0
  (99, 27835)	5.0
  (99, 27937)	14.0
  (99, 28034)	3.0
  (99, 28129)	14.0
  (99, 28232)	5.0
  (99, 28335)	20.0
  (99, 28438)	2.0
  (99, 28537)	13.0
  (99, 28629)	4.0
  (99, 28733)	7.0
  (99, 28834)	5.0
  (99, 28935)	13.0
  (99, 29030)	1.0
  (99, 29038)	1.0
  (99, 29134)	2.0
  (99, 29234)	7.0
  (99, 29334)	4.0
  (99, 29434)	7.0
  (99, 29537)	11.0
  (99, 29634)	2.0
  (99, 29732)	7.0
  (99, 29834)	2.0
  (99, 29929)	3.0
  (99, 30033)	9.0


In [17]:
# create a dictionary, i.e. mapping from 'word IDs' to 'words'
vocab = dict(zip(otu_ids, otus))
vocab

{0: u'OTU_360',
 1: u'OTU_5',
 2: u'OTU_47557',
 3: u'OTU_45150',
 4: u'OTU_3',
 5: u'OTU_2689',
 6: u'OTU_29537',
 7: u'OTU_1345',
 8: u'OTU_9',
 9: u'OTU_5851',
 10: u'OTU_53684',
 11: u'OTU_37081',
 12: u'OTU_57700',
 13: u'OTU_42126',
 14: u'OTU_57752',
 15: u'OTU_8',
 16: u'OTU_28173',
 17: u'OTU_233',
 18: u'OTU_64722',
 19: u'OTU_4',
 20: u'OTU_75815',
 21: u'OTU_2',
 22: u'OTU_79',
 23: u'OTU_2803',
 24: u'OTU_205',
 25: u'OTU_386',
 26: u'OTU_95',
 27: u'OTU_17',
 28: u'OTU_7',
 29: u'OTU_327',
 30: u'OTU_172',
 31: u'OTU_18',
 32: u'OTU_15',
 33: u'OTU_36',
 34: u'OTU_142',
 35: u'OTU_664',
 36: u'OTU_1',
 37: u'OTU_22',
 38: u'OTU_34',
 39: u'OTU_5929',
 40: u'OTU_330',
 41: u'OTU_46',
 42: u'OTU_56913',
 43: u'OTU_123',
 44: u'OTU_68612',
 45: u'OTU_17232',
 46: u'OTU_17793',
 47: u'OTU_5017',
 48: u'OTU_71422',
 49: u'OTU_197',
 50: u'OTU_7914',
 51: u'OTU_61214',
 52: u'OTU_63370',
 53: u'OTU_85',
 54: u'OTU_64499',
 55: u'OTU_68',
 56: u'OTU_47484',
 57: u'OTU_15460',
 5

In [18]:
n_samples = n
n_features = 1000 #m
n_components = 100 # number of topics

In [19]:
# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
"""
max_df: float in range [0,1], default=1.0. when building vocab ignore terms that have a 
    document freq strictly higher than the given threshold
min_df: float in range [0,1], default=1.0. ignore terms in doc with freq lower than given
    threshold
vocabulary: mapping or iterable, optional.Either a mapping (e.g. a dict) where keys are 
    terms and values are indices in the feature matrix, or an iterable over terms. if not given,
    a vocabulary is determined from the input documents.
"""

tf_vectorizer = CountVectorizer(max_df=1.0, min_df=0.0,max_features=n_features,
                               vocabulary=list(set(vocab)))
#tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
#                                max_features=n_features)
#t0 = time()
#tf = tf_vectorizer.fit_transform(data_samples)
#print("done in %0.3fs." % (time() - t0))
#print()

Extracting tf features for LDA...


In [20]:
help(tf_vectorizer)

Help on CountVectorizer in module sklearn.feature_extraction.text object:

class CountVectorizer(sklearn.base.BaseEstimator, VectorizerMixin)
 |  Convert a collection of text documents to a matrix of token counts
 |  
 |  This implementation produces a sparse representation of the counts using
 |  scipy.sparse.csr_matrix.
 |  
 |  If you do not provide an a-priori dictionary and you do not use an analyzer
 |  that does some kind of feature selection then the number of features will
 |  be equal to the vocabulary size found by analyzing the data.
 |  
 |  Read more in the :ref:`User Guide <text_feature_extraction>`.
 |  
 |  Parameters
 |  ----------
 |  input : string {'filename', 'file', 'content'}
 |      If 'filename', the sequence passed as an argument to fit is
 |      expected to be a list of filenames that need reading to fetch
 |      the raw content to analyze.
 |  
 |      If 'file', the sequence items must have a 'read' method (file-like
 |      object) that is called to fet

In [23]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([str(vocab[feature_names[i]])
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        #message += " ".join([str(feature_names[i])
        #                     for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [24]:
print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tf_vectorizer._validate_vocabulary()
tf_feature_names = tf_vectorizer.get_feature_names()

#print('tf_vectorizer.get_feature_names(): {0}'.
#  format(tf_vectorizer.get_feature_names()))

n_top_words = 20
print_top_words(lda, tf_feature_names, n_top_words)


Fitting LDA models with tf features, n_samples=100 and n_features=1000...
done in 13.757s.

Topics in LDA model:
Topic #0: OTU_37847 OTU_31409 OTU_74704 OTU_58532 OTU_63353 OTU_24139 OTU_49978 OTU_58890 OTU_61107 OTU_4153 OTU_73109 OTU_8855 OTU_2414 OTU_67825 OTU_54182 OTU_68518 OTU_5178 OTU_61143 OTU_9398 OTU_60040
Topic #1: OTU_40474 OTU_43658 OTU_13065 OTU_20512 OTU_5646 OTU_74889 OTU_34754 OTU_3430 OTU_3467 OTU_45005 OTU_21272 OTU_74457 OTU_36622 OTU_22882 OTU_3638 OTU_42370 OTU_58520 OTU_33680 OTU_20101 OTU_3325
Topic #2: OTU_47475 OTU_63145 OTU_27595 OTU_57844 OTU_63354 OTU_275 OTU_49617 OTU_38165 OTU_69425 OTU_58589 OTU_32447 OTU_28499 OTU_48222 OTU_6983 OTU_57859 OTU_15680 OTU_50635 OTU_4062 OTU_62747 OTU_8658
Topic #3: OTU_501 OTU_46256 OTU_75782 OTU_22874 OTU_4237 OTU_76107 OTU_275 OTU_60332 OTU_6724 OTU_70196 OTU_8658 OTU_63141 OTU_20106 OTU_6763 OTU_40657 OTU_68332 OTU_49271 OTU_55980 OTU_59039 OTU_31700
Topic #4: OTU_275 OTU_8658 OTU_68332 OTU_55980 OTU_75078 OTU_1436 OTU_

Topic #56: OTU_8658 OTU_275 OTU_28554 OTU_501 OTU_55980 OTU_75078 OTU_68332 OTU_1979 OTU_14200 OTU_46948 OTU_75889 OTU_70860 OTU_5380 OTU_70 OTU_46256 OTU_22874 OTU_1436 OTU_54217 OTU_61881 OTU_34315
Topic #57: OTU_74471 OTU_43389 OTU_53918 OTU_3706 OTU_56514 OTU_16461 OTU_60761 OTU_53671 OTU_66659 OTU_3623 OTU_62100 OTU_15913 OTU_26018 OTU_21852 OTU_63117 OTU_68072 OTU_2506 OTU_2456 OTU_56157 OTU_18184
Topic #58: OTU_275 OTU_28554 OTU_55980 OTU_68332 OTU_8658 OTU_23749 OTU_14200 OTU_75078 OTU_1979 OTU_1436 OTU_2147 OTU_38151 OTU_75889 OTU_46948 OTU_345 OTU_61881 OTU_70860 OTU_54217 OTU_3243 OTU_4775
Topic #59: OTU_71232 OTU_60071 OTU_67118 OTU_74070 OTU_72233 OTU_18977 OTU_275 OTU_52528 OTU_42964 OTU_56275 OTU_65379 OTU_1619 OTU_62249 OTU_3847 OTU_10089 OTU_47601 OTU_55980 OTU_60593 OTU_28554 OTU_8658
Topic #60: OTU_59664 OTU_75810 OTU_15116 OTU_34306 OTU_5179 OTU_275 OTU_67666 OTU_33350 OTU_7919 OTU_8658 OTU_52200 OTU_70316 OTU_19578 OTU_36766 OTU_55980 OTU_62481 OTU_68332 OTU_44097 

Next steps: validate (scatter plots of 'words' and 'topics'); test different parameters; scale up to larger sample size. Can also try NMF (but will need to do tf-idf transform).

To map taxa to geographic locations, need to map them to the samples that they are present in. The samples are then associated with geog coordinates.